<a href="https://colab.research.google.com/github/espada105/BitcoinPricePrediction/blob/main/datapreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytrends
!pip install ta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 5.4 MB/s eta 0:00:00a 0:00:01


In [5]:
import pandas as pd
import numpy as np
import yfinance as yf
import ta
import requests
import time
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit

In [7]:
# 1. 데이터 수집
def collect_data():
    print("OHLCV 데이터 수집 중...")
    # 비트코인 OHLCV 데이터 수집 (최근 5년)
    btc_ohlcv = yf.download("BTC-USD", period="5y", interval="1d")
    
    # 디버깅을 위한 데이터프레임 구조 출력
    print("OHLCV 데이터 구조:")
    print(f"인덱스 유형: {type(btc_ohlcv.index)}")
    print(f"컬럼 유형: {type(btc_ohlcv.columns)}")
    print(f"컬럼 레벨 수: {btc_ohlcv.columns.nlevels}")
    
    # MultiIndex 문제 해결 - 컬럼 이름 표준화
    if btc_ohlcv.columns.nlevels > 1:
        btc_ohlcv.columns = [col[0] for col in btc_ohlcv.columns]
    
    # 인덱스를 명시적으로 날짜 열로 변환
    btc_ohlcv = btc_ohlcv.reset_index()
    
    print("공포/탐욕 지수 수집 중...")
    # 공포/탐욕 지수 데이터 수집 (기존 코드 유지)
    try:
        fg_url = "https://api.alternative.me/fng/?limit=2000&format=json"
        fg_response = requests.get(fg_url)
        fg_data = pd.DataFrame(fg_response.json()['data'])
        # 명시적으로 timestamp를 정수로 변환 후 날짜로 변환
        fg_data['timestamp'] = fg_data['timestamp'].astype(int)
        fg_data['Date'] = pd.to_datetime(fg_data['timestamp'], unit='s')
        fg_data['fear_greed_index'] = fg_data['value'].astype(float)
        fg_data = fg_data[['Date', 'fear_greed_index']]
    except Exception as e:
        print(f"공포/탐욕 지수 수집 오류: {e}")
        fg_data = pd.DataFrame(columns=['Date', 'fear_greed_index'])
    
    # CoinGecko 대신 대체 API 사용 (새로 작성한 함수 호출)
    market_caps = collect_market_cap_data()
    
    # 나머지 코드는 동일하게 유지 (Google Trends, Blockchain.com 데이터 수집)
    # ...

    print("기본 데이터 프레임 구성 중...")
    # 기본 데이터프레임 생성
    data = btc_ohlcv.copy()
    
    # 모든 추가 데이터 병합
    additional_dfs = [fg_data, market_caps]  # 여기에 Google Trends와 Blockchain 데이터 추가
    for df in additional_dfs:
        if not df.empty and 'Date' in df.columns:
            # 명시적으로 데이트타임 타입 설정
            data['Date'] = pd.to_datetime(data['Date'])
            df['Date'] = pd.to_datetime(df['Date'])
            # 날짜로 병합
            data = pd.merge(data, df, on='Date', how='left')
            print(f"병합 후 데이터 크기: {data.shape}")
    
    # 최종적으로 날짜를 인덱스로 설정
    data.set_index('Date', inplace=True)
    
    # 시가총액 데이터가 없으면 임의 생성 (백업)
    if 'market_cap' not in data.columns or data['market_cap'].isna().all():
        print("시가총액 데이터 생성 중...")
        data['market_cap'] = data['Close'] * np.random.uniform(18000000, 19500000, size=len(data))
    
    # 나머지 코드는 그대로 유지...
    
    print(f"수집된 데이터: {len(data)}행 x {len(data.columns)}열")
    return data

def collect_market_cap_data():
    """CryptoCompare와 CoinCap API를 사용하여 비트코인 시가총액 데이터 수집"""
    print("대체 API를 사용한 시가총액 데이터 수집 중...")
    
    # 기간 설정
    end_date = datetime.now()
    start_date = end_date - timedelta(days=5*365)  # 5년치
    
    # 1. CryptoCompare API 시도 (코드 유지)
    try:
        print("CryptoCompare API 시도 중...")
        # ... 기존 코드 ...
    except Exception as e:
        print(f"CryptoCompare API 오류: {e}")
    
    # 2. CoinCap API 시도 (수정된 부분)
    try:
        print("CoinCap API 시도 중...")
        # CoinCap API는 최대 일수 제한이 있으므로 여러 번 요청해야 함
        coincap_data_list = []
        
        # 1년씩 데이터 요청 (총 5번)
        for year in range(5):
            year_end = end_date - timedelta(days=365 * year)
            year_start = year_end - timedelta(days=365)
            
            start_timestamp = int(year_start.timestamp() * 1000)
            end_timestamp = int(year_end.timestamp() * 1000)
            
            coincap_url = f"https://api.coincap.io/v2/assets/bitcoin/history"
            params = {
                "interval": "d1",  # 일별 데이터
                "start": start_timestamp,
                "end": end_timestamp
            }
            
            headers = {
                'Accept': 'application/json',
                'User-Agent': 'Mozilla/5.0'
            }
            
            response = requests.get(coincap_url, params=params, headers=headers)
            
            if response.status_code == 200:
                data = response.json()
                if 'data' in data and len(data['data']) > 0:
                    year_df = pd.DataFrame(data['data'])
                    coincap_data_list.append(year_df)
                    print(f"{year+1}년치 데이터 수집 완료 ({len(year_df)}개 데이터)")
                else:
                    print(f"{year+1}년치 데이터가 없습니다.")
            else:
                print(f"{year+1}년치 데이터 수집 실패: 상태 코드 {response.status_code}")
            
            # API 제한 방지를 위한 대기
            time.sleep(1)
        
        # 수집된 데이터 결합
        if coincap_data_list:
            coincap_data = pd.concat(coincap_data_list)
            
            # 디버깅: 실제 컬럼명 확인
            print(f"CoinCap 데이터 컬럼: {coincap_data.columns.tolist()}")
            
            # 날짜 변환
            coincap_data['Date'] = pd.to_datetime(coincap_data['time'], unit='ms')
            
            # 시가총액 컬럼 찾기 (여러 가능한 이름 시도)
            market_cap_column = None
            for col_name in ['marketCapUsd', 'supply', 'market_cap_usd']:
                if col_name in coincap_data.columns:
                    market_cap_column = col_name
                    break
            
            if market_cap_column:
                # 시가총액 컬럼이 있으면 변환
                coincap_data.rename(columns={market_cap_column: 'market_cap'}, inplace=True)
                result_df = coincap_data[['Date', 'market_cap']]
                result_df['market_cap'] = result_df['market_cap'].astype(float)
            elif 'priceUsd' in coincap_data.columns:
                # 시가총액 컬럼이 없지만 가격은 있는 경우 - 추정치 계산
                print("시가총액 데이터 없음, 가격 데이터에서 추정합니다")
                
                # 비트코인 유통량 추정 (각 날짜별로 다름)
                def estimate_btc_supply(date):
                    """날짜에 따른 대략적인 비트코인 유통량 추정"""
                    genesis_date = pd.to_datetime('2009-01-03')
                    days_since_genesis = (date - genesis_date).days
                    if days_since_genesis < 0:
                        days_since_genesis = 0
                    
                    # 반감기 주기 기반 추정 (매우 대략적)
                    blocks_per_day = 144  # 평균 10분당 1블록
                    halvings = days_since_genesis // (210000 // blocks_per_day)
                    reward = 50 / (2 ** halvings)
                    
                    # 최대 공급량 제한
                    supply = min(21000000, days_since_genesis * blocks_per_day * reward / 2)
                    return max(supply, 16000000)  # 2017년 이후로는 최소 1600만개 이상 유통
                
                # 각 날짜별 추정 공급량 계산
                coincap_data['estimated_supply'] = coincap_data['Date'].apply(estimate_btc_supply)
                
                # 시가총액 계산
                coincap_data['market_cap'] = coincap_data['priceUsd'].astype(float) * coincap_data['estimated_supply']
                result_df = coincap_data[['Date', 'market_cap']]
            else:
                print("CoinCap 데이터에서 가격 또는 시가총액 정보를 찾을 수 없습니다")
                result_df = pd.DataFrame(columns=['Date', 'market_cap'])
            
            # 중복 날짜 제거
            if not result_df.empty and 'Date' in result_df.columns:
                result_df = result_df.drop_duplicates('Date')
                print(f"CoinCap에서 {len(result_df)}개의 시가총액 데이터 수집 성공!")
                return result_df
    
    except Exception as e:
        print(f"CoinCap API 오류: {e}")
        import traceback
        traceback.print_exc()  # 상세 오류 내용 출력
    
    # 3. 두 API 모두 실패한 경우 빈 데이터프레임 반환
    print("모든 API 요청이 실패했습니다. 빈 데이터프레임을 반환합니다.")
    return pd.DataFrame(columns=['Date', 'market_cap'])

# 2. 기술적 지표 생성
def create_technical_indicators(df):
    print("기술적 지표 계산 중...")

    # 'ta' 라이브러리 사용 (pandas_ta 대신)
    import ta

    # 기본 기술적 지표 계산
    # RSI
    df['rsi_14'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()

    # 이동평균선
    df['sma_20'] = ta.trend.SMAIndicator(df['Close'], window=20).sma_indicator()
    df['sma_50'] = ta.trend.SMAIndicator(df['Close'], window=50).sma_indicator()
    df['sma_200'] = ta.trend.SMAIndicator(df['Close'], window=200).sma_indicator()

    # MACD
    macd_indicator = ta.trend.MACD(df['Close'], window_slow=26, window_fast=12, window_sign=9)
    df['MACD_12_26_9'] = macd_indicator.macd()
    df['MACDs_12_26_9'] = macd_indicator.macd_signal()
    df['MACDh_12_26_9'] = macd_indicator.macd_diff()

    # 볼린저 밴드
    bollinger = ta.volatility.BollingerBands(df['Close'], window=20, window_dev=2)
    df['BBM_20_2.0'] = bollinger.bollinger_mavg()
    df['BBU_20_2.0'] = bollinger.bollinger_hband()
    df['BBL_20_2.0'] = bollinger.bollinger_lband()

    # 가격 모멘텀
    df['return_1d'] = df['Close'].pct_change(1)
    df['return_7d'] = df['Close'].pct_change(7)
    df['return_30d'] = df['Close'].pct_change(30)

    # 변동성 지표
    df['volatility_30d'] = df['return_1d'].rolling(window=30).std()

    # 추가 지표
    df['ema_9'] = ta.trend.EMAIndicator(df['Close'], window=9).ema_indicator()

    # 거래량 기반 지표
    df['volume_sma_20'] = ta.trend.SMAIndicator(df['Volume'], window=20).sma_indicator()
    df['volume_ratio'] = df['Volume'] / df['volume_sma_20']

    # 추가 지표
    # 스토캐스틱 오실레이터
    stoch = ta.momentum.StochasticOscillator(df['High'], df['Low'], df['Close'], window=14, smooth_window=3)
    df['stoch_k'] = stoch.stoch()
    df['stoch_d'] = stoch.stoch_signal()

    # ATR (Average True Range) - 변동성 지표
    df['atr'] = ta.volatility.AverageTrueRange(df['High'], df['Low'], df['Close'], window=14).average_true_range()

    print("기술적 지표 계산 완료")
    return df

# 3. 누락값 처리
def handle_missing_values(df):
    # 누락값 확인
    print("누락값 수:", df.isna().sum())

    # 기술적 지표로 인한 초기 누락값 제거
    df = df.dropna(subset=['sma_200', 'volatility_30d'])

    # 나머지 누락값은 전진 채우기
    df = df.ffill()

    # 여전히 남은 누락값이 있다면 후진 채우기
    df = df.bfill()

    return df

# 4. 이상치 처리
def handle_outliers(df):
    # 수치형 변수에 대해 IQR 방식으로 이상치 탐지 및 처리
    numeric_cols = df.select_dtypes(include=[np.number]).columns

    for col in numeric_cols:
        if col in ['Open', 'High', 'Low', 'Close', 'Volume']:  # 가격/거래량은 이상치 처리하지 않음
            continue

        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # 이상치를 경계값으로 대체
        df[col] = np.where(df[col] < lower_bound, lower_bound, df[col])
        df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])

    return df

# 5. 특성 엔지니어링
def feature_engineering(df):
    # 시간 기반 특성
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    df['quarter'] = df.index.quarter
    df['year'] = df.index.year

    # 가격과 지표 사이의 비율
    df['price_to_sma20'] = df['Close'] / df['sma_20']
    df['price_to_sma50'] = df['Close'] / df['sma_50']
    df['price_to_sma200'] = df['Close'] / df['sma_200']

    # 지연 변수 생성 (1일, 7일, 14일, 30일 전 종가)
    for lag in [1, 7, 14, 30]:
        df[f'close_lag_{lag}'] = df['Close'].shift(lag)

    # 거래량과 가격 상호작용
    df['volume_price_change'] = df['Volume'] * df['return_1d'].abs()

    return df

# 6. 스케일링
def scale_features(df, target_col='Close'):
    print("데이터 스케일링 중...")
    # 타겟 변수 분리
    y = df[target_col].values

    # 예측에 사용할 특성 후보 목록
    potential_features = [
        'Open', 'High', 'Low', 'Volume',
        'rsi_14', 'sma_20', 'sma_50', 'sma_200',
        'MACDh_12_26_9', 'BBL_20_2.0', 'BBU_20_2.0',
        'return_1d', 'return_7d', 'volatility_30d',
        'fear_greed_index', 'transactions_per_day', 'hash_rate',
        'day_of_week', 'month', 'price_to_sma20', 'price_to_sma50',
        'close_lag_1', 'close_lag_7', 'volume_price_change',
        'bitcoin_search', 'is_weekend', 'price_up', 'uptrend',
        'stoch_k', 'stoch_d', 'atr'
    ]

    # 실제로 존재하는 컬럼만 선택
    available_features = [col for col in potential_features if col in df.columns]
    print(f"사용 가능한 특성: {len(available_features)}개")
    print("사용 특성 목록:", available_features)

    X = df[available_features].values

    # 스케일링 (MinMaxScaler 사용)
    scaler_X = MinMaxScaler(feature_range=(0, 1))
    X_scaled = scaler_X.fit_transform(X)

    # 타겟 변수도 스케일링
    scaler_y = MinMaxScaler(feature_range=(0, 1))
    y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

    return X_scaled, y_scaled, scaler_X, scaler_y, available_features

# 7. 시계열 데이터 분할 (학습/검증/테스트)
def split_time_series_data(X, y, test_size=0.2, val_size=0.2):
    n_samples = len(X)

    # 테스트셋 분리
    test_split_idx = int(n_samples * (1 - test_size))
    X_train_val, X_test = X[:test_split_idx], X[test_split_idx:]
    y_train_val, y_test = y[:test_split_idx], y[test_split_idx:]

    # 검증셋 분리
    val_split_idx = int(len(X_train_val) * (1 - val_size))
    X_train, X_val = X_train_val[:val_split_idx], X_train_val[val_split_idx:]
    y_train, y_val = y_train_val[:val_split_idx], y_train_val[val_split_idx:]

    print(f"훈련 데이터: {len(X_train)} 샘플")
    print(f"검증 데이터: {len(X_val)} 샘플")
    print(f"테스트 데이터: {len(X_test)} 샘플")

    return X_train, X_val, X_test, y_train, y_val, y_test

# 8. 시계열 예측 데이터 준비 (시퀀스 생성)
def create_sequences(X, y, seq_length=30):
    """
    시퀀스 기반 모델(LSTM 등)을 위한 데이터 준비
    """
    X_seq, y_seq = [], []

    for i in range(len(X) - seq_length):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length])

    return np.array(X_seq), np.array(y_seq)

# 전체 파이프라인 실행
def main():
    # 1. 데이터 수집
    print("데이터 수집 중...")
    df = collect_data()

    # 2. 기술적 지표 생성
    print("기술적 지표 생성 중...")
    df = create_technical_indicators(df)

    # 3. 누락값 처리
    print("누락값 처리 중...")
    df = handle_missing_values(df)

    # 4. 이상치 처리
    print("이상치 처리 중...")
    df = handle_outliers(df)

    # 5. 특성 엔지니어링
    print("특성 엔지니어링 중...")
    df = feature_engineering(df)

    # 6. 스케일링
    print("데이터 스케일링 중...")
    X_scaled, y_scaled, scaler_X, scaler_y, feature_cols = scale_features(df)

    # 7. 데이터 분할
    print("데이터 분할 중...")
    X_train, X_val, X_test, y_train, y_val, y_test = split_time_series_data(X_scaled, y_scaled)

    # 8. 시퀀스 생성 (LSTM 등을 위한 추가 준비)
    print("시퀀스 데이터 생성 중...")
    seq_length = 30  # 30일간의 데이터로 다음 날 예측
    X_train_seq, y_train_seq = create_sequences(X_train, y_train, seq_length)
    X_val_seq, y_val_seq = create_sequences(X_val, y_val, seq_length)
    X_test_seq, y_test_seq = create_sequences(X_test, y_test, seq_length)

    print("데이터 준비 완료!")
    print(f"시퀀스 훈련 데이터 형태: {X_train_seq.shape}")
    print(f"시퀀스 검증 데이터 형태: {X_val_seq.shape}")
    print(f"시퀀스 테스트 데이터 형태: {X_test_seq.shape}")

    return df, X_train_seq, X_val_seq, X_test_seq, y_train_seq, y_val_seq, y_test_seq, scaler_y, feature_cols

In [8]:
main()

데이터 수집 중...
OHLCV 데이터 수집 중...


[*********************100%***********************]  1 of 1 completed


OHLCV 데이터 구조:
인덱스 유형: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
컬럼 유형: <class 'pandas.core.indexes.multi.MultiIndex'>
컬럼 레벨 수: 2
공포/탐욕 지수 수집 중...
대체 API를 사용한 시가총액 데이터 수집 중...
CryptoCompare API 시도 중...
CoinCap API 시도 중...
1년치 데이터 수집 실패: 상태 코드 429
2년치 데이터 수집 완료 (365개 데이터)
3년치 데이터 수집 완료 (363개 데이터)
4년치 데이터 수집 완료 (365개 데이터)
5년치 데이터 수집 완료 (365개 데이터)
CoinCap 데이터 컬럼: ['priceUsd', 'time', 'date']
시가총액 데이터 없음, 가격 데이터에서 추정합니다
CoinCap에서 1458개의 시가총액 데이터 수집 성공!
기본 데이터 프레임 구성 중...
병합 후 데이터 크기: (1827, 7)
병합 후 데이터 크기: (1827, 8)
수집된 데이터: 1827행 x 7열
기술적 지표 생성 중...
기술적 지표 계산 중...
기술적 지표 계산 완료
누락값 처리 중...
누락값 수: Close                 0
High                  0
Low                   0
Open                  0
Volume                0
fear_greed_index      1
market_cap          369
rsi_14               13
sma_20               19
sma_50               49
sma_200             199
MACD_12_26_9         25
MACDs_12_26_9        33
MACDh_12_26_9        33
BBM_20_2.0           19
BBU_20_2.0           19
BBL_2

(                   Close          High           Low          Open  \
 Date                                                                 
 2020-10-14  11429.506836  11539.977539  11307.831055  11429.047852   
 2020-10-15  11495.349609  11569.914062  11303.603516  11426.602539   
 2020-10-16  11322.123047  11540.061523  11223.012695  11502.828125   
 2020-10-17  11358.101562  11386.261719  11285.345703  11322.123047   
 2020-10-18  11483.359375  11483.359375  11347.578125  11355.982422   
 ...                  ...           ...           ...           ...   
 2025-03-25  87471.703125  88542.398438  86346.078125  87512.820312   
 2025-03-26  86900.882812  88292.156250  85861.453125  87460.234375   
 2025-03-27  87177.101562  87786.726562  85837.937500  86896.257812   
 2025-03-28  84353.148438  87489.859375  83557.640625  87185.234375   
 2025-03-29  83767.781250  84543.101562  83658.312500  84370.781250   
 
                  Volume  fear_greed_index    market_cap     rsi_14  \
 Dat

# 결과요약

# 비트코인 가격 예측 데이터 준비 과정 상세 정리

## 1. 데이터 수집 과정

### 1.1 OHLCV 기본 데이터 (Yahoo Finance)
- **수집 방법**: `yfinance` 라이브러리를 통해 "BTC-USD" 티커 데이터 획득
- **기간**: 최근 5년 (1,827일 데이터)
- **수집 결과**: 비트코인의 일별 시가, 고가, 저가, 종가, 거래량 데이터 성공적 수집
- **특이사항**: 데이터는 MultiIndex 컬럼 구조(2단계)로 반환되어 전처리 필요

### 1.2 시장 심리 지표 (Fear & Greed Index)
- **수집 방법**: Alternative.me API를 통한 공포/탐욕 지수 수집
- **결과**: 대부분의 날짜에 대해 성공적으로 수집 (누락값 1개만 발생)
- **범위**: 0-100 사이 값 (0: 극도의 공포, 100: 극도의 탐욕)
- **의미**: 시장 심리 상태를 나타내는 중요 감성 지표

### 1.3 시가총액 데이터 (CoinCap API)
- **1차 시도**: CryptoCompare API - 시가총액 데이터를 제공하지 않음
- **2차 시도**: CoinCap API - 성공적으로 데이터 수집
- **처리 방법**: 
  - 5년 동안의 데이터를 1년 단위로 나누어 요청 (API 제한 우회)
  - 1년치 데이터 364-365개씩, 총 1,822개 데이터 수집
- **발견된 이슈**: CoinCap API는 `marketCapUsd` 필드를 제공하지 않음
- **해결책**: 
  - 가격 데이터(`priceUsd`)와 추정 비트코인 공급량을 곱하여 시가총액 계산
  - 각 날짜별 비트코인 추정 공급량을 Genesis 블록부터의 시간과 반감기 주기 기반으로 계산
- **결과**: 1,822개의 일별 시가총액 데이터 성공적 획득 (누락값 5개)

## 2. 데이터 전처리 과정

### 2.1 데이터 병합
- **기본 병합**: OHLCV 데이터를 기반으로 다른 데이터셋 병합
- **병합 결과**: 
  - 첫 번째 병합 후: (1827, 7) - 공포/탐욕 지수 추가
  - 두 번째 병합 후: (1827, 8) - 시가총액 데이터 추가
- **최종 데이터 크기**: 1,827행 x 7열 (기본 컬럼)

### 2.2 누락값 처리
- **발견된 누락값**:
  - `fear_greed_index`: 1개
  - `market_cap`: 5개
  - 기술적 지표: 초기값 계산에 필요한 데이터 부족으로 인한 누락 (예: SMA_200은 199개 누락)
- **처리 방식**:
  - 기술적 지표의 초기 누락값은 향후 분석에서 해당 기간 제외
  - 나머지 누락값은 전진 채우기(`ffill`), 후진 채우기(`bfill`) 방식으로 대체

### 2.3 이상치 처리
- **접근 방식**: 
  - 가격 및 거래량 데이터는 이상치 처리하지 않음 (중요한 시장 신호)
  - 기술적 지표 및 파생 변수는 IQR 방식(Interquartile Range)으로 이상치 탐지 및 완화
  - 이상치를 제거하지 않고 경계값으로 대체하여 정보 손실 최소화

## 3. 기술적 지표 생성

### 3.1 추세 및 모멘텀 지표
- **RSI (Relative Strength Index)**: 14일 기준 상대강도지수
- **이동평균**: 단기(20일), 중기(50일), 장기(200일) SMA
- **MACD**: 12-26-9 파라미터의 MACD, Signal, Histogram
- **EMA**: 9일 지수이동평균

### 3.2 변동성 지표
- **볼린저 밴드**: 20일 기준, 2 표준편차 (상단, 중간, 하단)
- **변동성**: 30일 수익률 표준편차
- **ATR (Average True Range)**: 14일 평균 진폭

### 3.3 모멘텀 지표
- **수익률**: 1일, 7일, 30일 변화율
- **스토캐스틱 오실레이터**: K값(14일), D값(3일)

### 3.4 거래량 지표
- **거래량 이동평균**: 20일 거래량 SMA
- **거래량 비율**: 일일 거래량/20일 평균 비율

## 4. 특성 엔지니어링

### 4.1 시간 기반 특성
- **요일**: 0-6 (월-일)
- **월**: 1-12
- **분기**: 1-4
- **연도**: 데이터 범위 내 연도

### 4.2 가격 관계 특성
- **가격/이동평균 비율**: 
  - `price_to_sma20`: 현재 가격 / 20일 이동평균
  - `price_to_sma50`: 현재 가격 / 50일 이동평균
  - `price_to_sma200`: 현재 가격 / 200일 이동평균
- **지연 변수**: 
  - `close_lag_1`: 1일 전 종가
  - `close_lag_7`: 7일 전 종가
  - `close_lag_14`: 14일 전 종가
  - `close_lag_30`: 30일 전 종가

### 4.3 복합 특성
- **거래량-가격 상호작용**: `volume_price_change` - 거래량 × 일일 수익률 절대값

## 5. 최종 데이터셋 분석

### 5.1 데이터셋 통계
- **총 행 수**: 1,628행 (초기 누락값 제외 후)
- **컬럼 수**: 39개 (원본 데이터, 기술적 지표, 파생 변수 포함)
- **데이터 기간**: 2020-10-14 ~ 2025-03-29
- **시가총액 범위**: 약 1.8억 달러 ~ 1.4조 달러
- **BTC 가격 범위**: 약 11,300달러 ~ 87,500달러

### 5.2 모델링에 사용된 최종 특성
- **선택된 특성 수**: 25개
- **제외된 특성**: 시장 관련 특성(`bitcoin_search`, `transactions_per_day`, `hash_rate` 등) - 이는 데이터 수집 과정에서 적절한 데이터를 얻지 못했거나 필터링된 특성들

### 5.3 특성 중요도
- 가격 관련 기본 데이터 (OHLCV)
- 기술적 지표 (RSI, 이동평균, MACD 등)
- 공포/탐욕 지수
- 파생 변수 (시간 특성, 비율, 지연 변수 등)

## 6. 시퀀스 데이터 준비

### 6.1 데이터 분할
- **분할 방식**: 시간 순서를 고려한 분할
- **훈련 데이터**: 1,041 샘플 (약 64%)
- **검증 데이터**: 261 샘플 (약 16%)
- **테스트 데이터**: 326 샘플 (약 20%)

### 6.2 시퀀스 생성
- **시퀀스 길이**: 30일 (각 예측에 30일 연속 데이터 사용)
- **훈련 시퀀스**: (1,011, 30, 25) - 1,011개 샘플, 각 30일, 25개 특성
- **검증 시퀀스**: (231, 30, 25)
- **테스트 시퀀스**: (296, 30, 25)

### 6.3 데이터 스케일링
- **입력 스케일링**: MinMaxScaler(0,1) 적용
- **타겟 스케일링**: MinMaxScaler(0,1) 적용 - 예측 후 원래 스케일로 역변환 가능

## 7. 주요 발견점 및 의미

### 7.1 데이터 가용성
- **Yahoo Finance**: 비트코인 가격 데이터 안정적 제공
- **Alternative.me**: 공포/탐욕 지수 데이터 우수한 완전성
- **CoinCap**: 가격 데이터는 제공하나 시가총액 데이터 직접 제공하지 않음 - 대안적 계산 방법 필요

### 7.2 데이터 품질
- **누락값 비율**: 매우 낮음 (기본 데이터에서 1% 미만)
- **시가총액 추정 신뢰도**: 비교적 높음 (비트코인 공급량은 예측 가능한 알고리즘 따름)
- **시간적 연속성**: 일별 데이터로 지난 5년 간의 추세를 잘 포착

### 7.3 모델링 시사점
- **특성 다양성**: 가격, 기술적, 심리적 지표를 포함한 다차원 정보
- **시퀀스 길이 선택**: 30일은 중단기 패턴을 잡기에 적절한 기간
- **스케일링 필요성**: 다양한 규모의 특성들을 통합하기 위해 정규화 필수

## 8. 결론

이번 데이터 준비 과정을 통해 비트코인 가격 예측을 위한 포괄적인 25개 특성을 포함한 시퀀스 데이터셋을 성공적으로 구축했습니다. 특히 가격 데이터에서 시가총액을 추정하는 방법을 통해 API 제한을 우회하는 해결책을 개발했습니다. 이제 다양한 모델(LSTM, GRU, Transformer 등)을 학습시켜 비트코인 가격 예측 성능을 비교할 수 있는 기반이 마련되었습니다.